# 585.783 Introduction to Brain Computer Interfaces
## Mod 9 - Brain-Computer Interfacing – Introduction to Brain Computer Interfaces

### Date Submitted: August, DAY, 2022

### Eitan Waks

Answer the following questions using the robintibor/high-gamma-dataset – specifically data/train/1.mat.  Make sure you provide a notebook or source code as an appendix to this assignment.  Again, make sure all plots have labeled axes and meaningful titles.

Hint: You should be able to copy paste and re-use sections from the lecture notebooks provided in this module to complete this assignment with very little coding.

#### Q1.  **Outperform the professor.**

Your only responsibility for this homework is to build a neural classifier for the provided dataset that outperforms the one we presented in lecture.  You are free to just jump in and start modifying the lecture notebooks to derive a better result on the four-class classification problem.

***Please continue to use leave-one-out cross-validation*** when reporting your classification results.  You will be graded on substantive changes to the analysis that result in substantive performance changes in classification. Simply changing the random seeds used in the classifier methods, you might randomly get a slightly better classification performance, but this would not constitute a meaningful change in the analysis

When I ran the four-class problem with leave-one out classification, I achieved 50% accuracy.  I would consider performance >= 65% to be a substantive change in accuracy.  Remember that chance performance for a 4-class problem is 25%.

Things you might try:
- Feature Engineering
    - New signal processing (maybe multi-taper spectrograms?)
    - Sub-select new frequencies 
    (Hint: there might be some useful content between 70 and 80 Hz ;) )
    - Sub-select channels, or possibly different time-periods w.r.t. trial onset.
- New Classifier
    - Try an SVM classifier, or a logistic regression.  Random forests?
- Outlier Rejection
    - There might be some bad trials mixed into the dataset, try manually cleaning the dataset


To assess your performance in this homework, I would like to see the following content in a short report (1-2 pages).
1.	Description of task and trial types (1, 2, 3, 4) and what the subject was actually doing during these trial types. (You can get this from reading the paper associated from the dataset, https://onlinelibrary.wiley.com/doi/full/10.1002/hbm.23730 or by examining the topographic spectrograms) [4 pts]
2.	2-3 paragraph description of your change to the analysis and what changes you made to get better performance [8 pts]
3.	Confusion plot showing your classifier’s performance across different conditions.  Provide a short description of what the confusion plot means, and which trial types are being confused with each-other. 
See https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html for more information on confusion plots. [4pts]
4.	Model Introspection: Please provide a plot or two and a 1 paragraph description of what frequencies, channels, post-trial timings are contributing to your model’s performance. [4pts] (NB: to get these 4 points, you should probably pick a model that lends itself well to introspection – deep learning may not be what you want to use for this homework, but if you think you can successfully introspect and explain the deep network, be my guest.)


In [1]:
# imports
import mne
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt


In [5]:
# Paths
dataset_name = '1'
dataset_filetype = 'edf'
repository = 'BCI_MOD9_HW'
app_dir = Path('/', 'app')
data_dir = app_dir.joinpath('data')
notebook_dir = app_dir.joinpath(repository)
data_file = data_dir.joinpath(f'{dataset_name}.{dataset_filetype}')

In [ ]:
# Set up Montage -- Channel names in this file adhere to 10-05
montage = mne.channels.make_standard_montage('standard_1005')

In [ ]:
# Load the dataset
chs_EOG = ['EOG EOGh', 'EOG EOGv']
chs_EMG = ['EMG EMG_RH', 'EMG EMG_LH', 'EMG EMG_RF']
raw = mne.io.read_raw_edf(data_file, eog=chs_EOG, misc=chs_EMG, stim_channel='auto', exclude=(), infer_types=False, preload=True, verbose=None)
original_raw = raw.copy()
# Get events from the database
annotations = raw.annotations
events_from_annot, event_dict = mne.events_from_annotations(raw)
# epoch surrounding trial onset.
fs = raw.info['sfreq']
starttime = -3
endtime = 3
timebase_samp = np.arange(int(fs*starttime), int(fs*endtime))
timebase_sec = timebase_samp/fs

In [ ]:
# re-reference to the common average using MNE
raw_car = raw.copy().set_eeg_reference('average')

In [ ]:
# Resample
raw_resampled = raw_car.copy().resample()

In [ ]:
# Bandpass filter
lowpass_freq = 1
highpass_freq = 30
raw_filtered = raw_car.copy().filter(l_freq=lowpass_freq, h_freq=highpass_freq)

In [ ]:
# Set list of all channels whose name contains the string “AF” and the corresponding indeces
AF_ch = [ch for ch in raw.ch_names if 'AF' in ch]
raw_filtered_AF = raw_filtered.copy().pick(AF_ch)  # selects only the EEG channels

In [ ]:
# plot all channels whose name contains the string “AF”. 
# A 10 second stretch beginning at 200 seconds into the file
starttime = 200
duration = 10
fig01 = raw_filtered_AF.plot(events=None, duration=duration, start=starttime, n_channels=len(AF_ch), 
    title=f'EEG of AF channels, common average referenced and bandpass filtered between 1 and 30 Hz, beginning at {starttime} seconds for a duration of {duration} seconds', 
    show_scrollbars=False, show_scalebars=True, time_format='float')
fig01.show()
fig01.savefig(f'AF_EEG_{starttime}_{duration}.png', dpi=100, format='png', metadata={'Title': f'AF_EEG_{starttime}_{duration}', 'Author': 'Eitan Waks', 
            'Description': f'EEG plot of {AF_ch} channels, preprocessed with common average reference and bandpass filtered between 1 and 30 Hz, starting at {starttime} seconds for a duration of {duration} seconds. The data is from the robintibor/high-gamma-dataset (data/train/1.edf)'},
            bbox_inches=None, pad_inches=0.1, facecolor='auto', edgecolor='auto', backend=None)

In [ ]:
# Set list of channels and corresponding indeces. Get channel data
chs = ['EEG Cz', 'EEG C3', 'EEG C4', 'EEG Pz', 'EEG Oz']
ch_idxs = [raw.ch_names.index(ch) for ch in chs]
raw_filtered_chs = raw_filtered.copy().pick(chs)  # selects only the 'EEG Cz', 'EEG C3', 'EEG C4', 'EEG Pz', 'EEG Oz' channels
raw_filtered_chs_data, raw_filtered_chs_t = raw_filtered_chs.copy()[:, :]

In [ ]:
# Create directory of events according to event type
idxs = list(range(0, len(events_from_annot), 1))
events = event_dict.copy()
events['feet'] = [events_from_annot[idx][0] for idx in idxs if events_from_annot[idx][2]==event_dict['feet']]
events['left_hand'] = [events_from_annot[idx][0] for idx in idxs if events_from_annot[idx][2]==event_dict['left_hand']]
events['rest'] = [events_from_annot[idx][0] for idx in idxs if events_from_annot[idx][2]==event_dict['rest']]
events['right_hand'] = [events_from_annot[idx][0] for idx in idxs if events_from_annot[idx][2]==event_dict['right_hand']]

In [ ]:
# Create dictionary of event epochs according to type
events_epochs = event_dict.copy()
events_epochs['feet'] = [events['feet'][idx] + timebase_samp for idx in list(range(0, len(events['feet']), 1))]
events_epochs['left_hand'] = [events['left_hand'][idx] + timebase_samp for idx in list(range(0, len(events['left_hand']), 1))]
events_epochs['rest'] = [events['rest'][idx] + timebase_samp for idx in list(range(0, len(events['rest']), 1))]
events_epochs['right_hand'] = [events['right_hand'][idx] + timebase_samp for idx in list(range(0, len(events['right_hand']), 1))]

In [ ]:
# Create dictionary of event values split by trial type and epoch
events_epochs_values = event_dict.copy()
events_epochs_values['feet'] = [raw_filtered_chs_data[:,events_epochs['feet'][idx]] for idx in list(range(0, len(events_epochs['feet']), 1))]
events_epochs_values['left_hand'] = [raw_filtered_chs_data[:,events_epochs['left_hand'][idx]] for idx in list(range(0, len(events_epochs['left_hand']), 1))]
events_epochs_values['rest'] = [raw_filtered_chs_data[:,events_epochs['rest'][idx]] for idx in list(range(0, len(events_epochs['rest']), 1))]
events_epochs_values['right_hand'] = [raw_filtered_chs_data[:,events_epochs['right_hand'][idx]] for idx in list(range(0, len(events_epochs['right_hand']), 1))]

In [ ]:
for ch_idx, ch in enumerate(chs):
    fig, axs = plt.subplots( nrows=2, ncols=2, dpi=100, figsize=(9.0,6.0), constrained_layout=True, sharex=True, sharey=True)
    fig.suptitle(f'{ch} Trial Average ERP', fontsize=16)
    fig.supxlabel('Time (sec) relative to trial onset')
    fig.supylabel('Voltage (uV)')
    for nn, ax in enumerate(axs.flat):
        line_width = 2.0
        ax.set_xlim(-1.0, 3.0)
        ax.set_ylim([-19.0, 10.0])
        ax.grid()
        column = list(event_dict.copy().keys())[nn]
        ax.set_title(column, fontsize='small', loc='center')
        for event_type_idx, event_type in enumerate(event_dict):
            erp = events_epochs_values[event_type][:][ch_idx][:].mean(axis=0)
            erp = erp*1e6
            
            if nn == 0 and event_type == "left_hand":
                ax.plot(timebase_sec, erp, lw=line_width)
                break;
            elif nn == 1 and event_type == "right_hand":
                ax.plot(timebase_sec, erp, lw=line_width)
                break;
            elif nn == 2 and event_type == "feet":
                ax.plot(timebase_sec, erp, lw=line_width)
                break;
            elif nn == 3 and event_type == "rest":
                ax.plot(timebase_sec, erp, lw=line_width)
                break;
            else:
                pass
    plt.show()
    plt.savefig(f'{ch}.png', dpi=100, format='png', metadata={'Title': f'{ch}', 'Author': 'Eitan Waks', 
            'Description': f'The trial average ERP for EEG channel {ch} for all trial types from the robintibor/high-gamma-dataset (data/train/1.edf)'},
            bbox_inches=None, pad_inches=0.1, facecolor='auto', edgecolor='auto', backend=None
       )
